In [5]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import math

In [3]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.2)

def resize_image(image, size=(224, 224)):
    return cv2.resize(image, size, interpolation=cv2.INTER_AREA)

# Function to rotate image based on angle
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    
    # Calculate the rotation matrix
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # Calculate the new bounding dimensions of the image
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))
    
    # Adjust the rotation matrix to account for the translation
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]
    
    # Perform the actual rotation and return the image
    rotated = cv2.warpAffine(image, M, (new_w, new_h))
    return rotated

# Function to get the rotation angle to make the palm facing up
def get_rotation_angle(landmarks):
    # Using the wrist (0) and middle finger MCP (9) landmarks to determine the angle
    x1, y1 = landmarks[0].x, landmarks[0].y
    x2, y2 = landmarks[9].x, landmarks[9].y
    angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
    return angle + 90

# Function to crop the hand region from the image
def crop_hand(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Deteksi tangan dalam gambar
    results = hands.process(image_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Dapatkan koordinat bounding box dari telapak tangan
            h, w, _ = image.shape
            selected_landmarks = [0, 1, 5, 9, 13, 17]  # Landmarks yang mewakili telapak tangan
            
            x_min = int(min([hand_landmarks.landmark[i].x for i in selected_landmarks]) * w)
            y_min = int(min([hand_landmarks.landmark[i].y for i in selected_landmarks]) * h)
            x_max = int(max([hand_landmarks.landmark[i].x for i in selected_landmarks]) * w)
            y_max = int(max([hand_landmarks.landmark[i].y for i in selected_landmarks]) * h)
            
            # Tambahan margin jika diperlukan
            margin = 10
            x_min = max(0, x_min - margin)
            y_min = max(0, y_min - margin)
            x_max = min(w, x_max + margin)
            y_max = min(h, y_max + margin)
            
            # Crop gambar
            cropped_image = image[y_min:y_max, x_min:x_max]

            # Convert the image to grayscale
            gray_img = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
                    
            # Apply Gaussian Blur
            blurred_img = cv2.GaussianBlur(gray_img, (7, 7), 0)

            return blurred_img

# Function to process and detect hand in images
def process_and_detect(folder_path, output_rotate_dir, output_crop_dir):
    not_detected_files = []
    detected_images = []

    # Iterate through each image in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(('.JPG', '.jpg')):
                image_path = os.path.join(folder_path, filename)
                image = cv2.imread(image_path)
                if image is None:
                    continue
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = hands.process(image_rgb)
                
                if results.multi_hand_landmarks:
                    landmarks = results.multi_hand_landmarks[0].landmark
                    rotation_angle = get_rotation_angle(landmarks)

                    # Rotate the image to make the palm face up
                    rotated_image = rotate_image(image, rotation_angle)

                    # Save the rotated image
                    output_dir = os.path.join(output_rotate_dir, os.path.basename(folder_path))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    output_path = os.path.join(output_dir, filename)
                    cv2.imwrite(output_path, rotated_image)

                    # Crop the hand region
                    cropped_image = crop_hand(rotated_image)

                    # Save the cropped image
                    output_crop_dir_path = os.path.join(output_crop_dir, os.path.basename(folder_path))
                    if not os.path.exists(output_crop_dir_path):
                        os.makedirs(output_crop_dir_path)
                    output_crop_path = os.path.join(output_crop_dir_path, filename)
                    cv2.imwrite(output_crop_path, cropped_image)

                    # Resize the image
                    resized_img = resize_image(cropped_image)
                    
                    # Save the resized image
                    resized_image = resize_image(cropped_image)
                    output_resize_dir_path = os.path.join(output_resize_dir, os.path.basename(folder_path))
                    if not os.path.exists(output_resize_dir_path):
                        os.makedirs(output_resize_dir_path)
                    output_resize_path = os.path.join(output_resize_dir_path, filename)
                    cv2.imwrite(output_resize_path, resized_image)

                    detected_images.append(resized_image)
                else:
                    not_detected_files.append(filename)

    return not_detected_files

# Path to the dataset directory
dataset_path = r"C:\Users\alfin\Skripsi\Dataset\Birjand University Mobile Palmprint Database (BMPD)"
output_rotate_dir = r"C:\Users\alfin\Skripsi\Dataset\Rotate"
output_crop_dir = r"C:\Users\alfin\Skripsi\Dataset\Crop"
output_resize_dir = r"C:\Users\alfin\Skripsi\Dataset\Resize"

# Process each ID folder
for id_folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, id_folder)
    if os.path.isdir(folder_path):
        process_and_detect(folder_path, output_rotate_dir, output_crop_dir)

# Clean up
hands.close()


In [4]:
# Function to calculate LBP
def calculate_lbp(image):
    height, width = image.shape
    lbp_image = np.zeros((height, width), dtype=np.uint8)
    neighbors = [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)]
    
    for y in range(1, height-1):
        for x in range(1, width-1):
            center = image[y, x]
            binary_string = ''
            
            for dy, dx in neighbors:
                neighbor_value = image[y + dy, x + dx]
                binary_string += '1' if neighbor_value >= center else '0'
            
            lbp_value = int(binary_string, 2)
            lbp_image[y, x] = lbp_value
    
    return lbp_image

# Function to process LBP for images in a folder
def process_lbp_images(input_dir, output_lbp_dir):
    if not os.path.exists(output_lbp_dir):
        os.makedirs(output_lbp_dir)

    for id_folder in os.listdir(input_dir):
        folder_path = os.path.join(input_dir, id_folder)
        output_folder_path = os.path.join(output_lbp_dir, id_folder)

        if not os.path.exists(output_folder_path):
            os.makedirs(output_folder_path)

        for filename in os.listdir(folder_path):
            if filename.endswith(('.JPG', '.jpg')):
                image_path = os.path.join(folder_path, filename)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                if image is None:
                    continue
                
                lbp_image = calculate_lbp(image)
                output_path = os.path.join(output_folder_path, filename)
                cv2.imwrite(output_path, lbp_image)

# Paths
intput_resize_dir = r"C:\Users\alfin\Skripsi\Dataset\Resize"
output_lbp_dir = r"C:\Users\alfin\Skripsi\Dataset\LBP"

# Process LBP for each cropped image
process_lbp_images(intput_resize_dir, output_lbp_dir)
